In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_excel("dataWithSomeReformatting.xlsx", index_col = [0])
keys = list(data.keys())
keys

['Article_title',
 'Article_link',
 'General_outcome',
 'General_outcome (1,0)',
 'Desc_general_outcome',
 'Daily_intake',
 'Descr_daily_intake',
 'Health_outcome',
 'Descr_health_outcome',
 'Environmental_outcome',
 'Descr_environmental_outcome',
 'Fund_source',
 'Fund_Source',
 'Author_affiliation',
 'Loc_authors',
 'Loc_experiment',
 'Year',
 'Journal',
 'Loc_journal',
 'Loc_editor_journal',
 'Current Impact_Factor',
 'Impact_F_Publishing_Year',
 'Num_Citations',
 'Public',
 'Private',
 'University',
 'International',
 'Research',
 'MEXICO',
 'ARGENTINA',
 'UKRAINE',
 'LATVIA',
 'GERMANY',
 'NEW ZEALAND',
 'BRAZL',
 'NETHERLANDS',
 'NIGERIA',
 'SWEDEN',
 'FRANCE',
 'PORTUGAL',
 'CROATIA',
 'SLOVENIA',
 'SPAIN',
 'POLAND',
 'ITALY',
 'ECUADOR',
 'MOROCCO',
 'GREECE',
 'INDIA',
 'IRAQ',
 'UK',
 'JAPAN',
 'ROMANIA',
 'CHINA',
 'IRAN',
 'IRELAND',
 'AUSTRIA',
 'TUNISIA',
 'SLOVAK REPUBLIC',
 'FINLAND',
 'TAIWAN',
 'AUSTRALIA',
 'TURKEY',
 'REPUBLIC OF KOREA',
 'SRI LANKA',
 'SOUTH AFRIC

In [3]:
years = keys[-31:]
countries = keys[-82:-31]
data.rename(columns = {year:str(year) for year in years}, inplace = True)
years = [str(year) for year in years]

In [4]:
# drop Iran due to colinearity
#del years[0]
keys =['Current Impact_Factor',
 'Num_Citations',
 'Public',
 'Private',
 'University',
 'International',
 'Research']
#keys = ["General_outcome", "Year", "Fund_Source", "Loc_experiment", "Journal"]
max_cond_vars = len(keys) - 1
dag_keys = keys #+ years #+ countries
data[dag_keys]

,Current Impact_Factor,Num_Citations,Public,Private,University,International,Research
0,2.790,1.0,1,0,0,0,0
1,1.812,29.0,1,0,0,0,0
2,0.680,31.0,0,0,0,0,0
3,2.950,20.0,0,1,0,0,0
4,4.679,108.0,1,0,0,0,0
5,2.600,29.0,1,0,0,0,0
6,2.504,22.0,1,0,0,0,0
7,2.552,31.0,1,0,0,0,0
8,NaN,13.0,0,0,0,1,0
9,0.000,3.0,0,0,0,0,0


In [5]:
import pingouin

def gen_pcorr(df, method = "pearson", sig = 0.01):
    # Correlation type:
    # 'pearson': Pearson r product-moment correlation
    # 'spearman': Spearman ρ rank-order correlation
    # 'kendall': Kendall’s τB correlation (for ordinal data)
    # 'bicor': Biweight midcorrelation (robust)
    # 'percbend': Percentage bend correlation (robust)
    # 'shepherd': Shepherd’s pi correlation (robust)
    # 'skipped': Skipped correlation (robust)
    pcs_dct = {}
    sig_corr_dct = {}
    for x in df.keys():
        sig_corr_dct[x] = []
        pcs_dct[x]={}
        for y in df.keys():
            # control variables
            # select variables that are not x or y
            other_vars = [z for z in df.keys() if z != y and z != x ]
            if x == y:
                # No need to calculate if the variable is itself
                pcs_dct[x][y] = 1
            else:
                pcs_dct[x][y] = df.partial_corr(x=x,y=y, covar=other_vars,
                                      method=method).round(3)
                if pcs_dct[x][y]["p-val"].values[0] < sig:
                    sig_corr_dct[x].append((y, pcs_dct[x][y]["r"].values[0]))

    return pcs_dct, sig_corr_dct

In [6]:
pcs_dct, sig_corr_dct = gen_pcorr(data[dag_keys].fillna(0), method = "pearson", sig = 0.01)

import statsmodels.api as sm
import numpy
residuals = {}
partial_corr = {}
reg_df = data[dag_keys].dropna()[:-1]
for y_var in dag_keys:
    X_vars = [x for x in dag_keys if x != y_var]
    X= reg_df[X_vars]
    X["constant"] = 1
    y = reg_df[y_var]
    model = sm.OLS(y,X)
    results = model.fit()
    print(results.summary())
    predict = results.predict()
    reg_df["predict"] = predict
    residuals[y_var] = results.resid
r2 = {}
for x in dag_keys:
    partial_corr[x] = {}
    r2[x] = {}
    for y in dag_keys:
        if x != y:
            Y = pd.DataFrame(residuals[y])
            X = pd.DataFrame(residuals[x])
            model = sm.OLS(Y,X)
            results = model.fit()
            print(results.rsquared, results.pvalues)
            
            # check if statsmodels is correctly calcualting the error term... May be negative
            partial_corr[x][y] = np.corrcoef(residuals[x], residuals[y])[0][1] * -1
            print(partial_corr[x][y])
            print(pcs_dct[x][y])
        else:
            partial_corr[x][y] = 1
pd.DataFrame(partial_corr).sort_index(axis=0, ascending=True).sort_index(axis=1, ascending = True) 

                              OLS Regression Results                             
Dep. Variable:     Current Impact_Factor   R-squared:                       0.100
Model:                               OLS   Adj. R-squared:                  0.089
Method:                    Least Squares   F-statistic:                     8.975
Date:                   Mon, 12 Jul 2021   Prob (F-statistic):           2.62e-09
Time:                           22:52:12   Log-Likelihood:                -952.50
No. Observations:                    490   AIC:                             1919.
Df Residuals:                        483   BIC:                             1948.
Df Model:                              6                                         
Covariance Type:               nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Num_Citations   

<ipython-input-6-249c743f975e>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  X["constant"] = 1
<ipython-input-6-249c743f975e>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  X["constant"] = 1
<ipython-input-6-249c743f975e>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  X["constant"] = 1


                            OLS Regression Results                            
Dep. Variable:                Private   R-squared:                       0.085
Model:                            OLS   Adj. R-squared:                  0.074
Method:                 Least Squares   F-statistic:                     7.493
Date:                Mon, 12 Jul 2021   Prob (F-statistic):           1.08e-07
Time:                        22:52:12   Log-Likelihood:                -83.528
No. Observations:                 490   AIC:                             181.1
Df Residuals:                     483   BIC:                             210.4
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Current Impact_Factor     0.02

<ipython-input-6-249c743f975e>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  X["constant"] = 1
<ipython-input-6-249c743f975e>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  X["constant"] = 1
<ipython-input-6-249c743f975e>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  X["constant"] = 1


                            OLS Regression Results                            
Dep. Variable:               Research   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                 -0.004
Method:                 Least Squares   F-statistic:                    0.6353
Date:                Mon, 12 Jul 2021   Prob (F-statistic):              0.702
Time:                        22:52:13   Log-Likelihood:                 556.62
No. Observations:                 490   AIC:                            -1099.
Df Residuals:                     483   BIC:                            -1070.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Current Impact_Factor     0.00

,Current Impact_Factor,International,Num_Citations,Private,Public,Research,University
Current Impact_Factor,1.000000,0.130862,0.206926,0.123330,0.172740,0.030260,0.078119
International,0.130862,1.000000,-0.014639,-0.092353,-0.142811,-0.026772,-0.072414
Num_Citations,0.206926,-0.014639,1.000000,0.123709,0.013922,0.019019,0.025259
Private,0.123330,-0.092353,0.123709,1.000000,-0.173587,-0.048557,-0.169160
Public,0.172740,-0.142811,0.013922,-0.173587,1.000000,-0.061295,-0.049959
Research,0.030260,-0.026772,0.019019,-0.048557,-0.061295,1.000000,-0.053085
University,0.078119,-0.072414,0.025259,-0.169160,-0.049959,-0.053085,1.000000


In [7]:
reg_df[dag_keys].pcorr().sort_index(axis=0, ascending=True).sort_index(axis=1, ascending = True)

,Current Impact_Factor,International,Num_Citations,Private,Public,Research,University
Current Impact_Factor,1.000000,0.130862,0.206926,0.123330,0.172740,0.030260,0.078119
International,0.130862,1.000000,-0.014639,-0.092353,-0.142811,-0.026772,-0.072414
Num_Citations,0.206926,-0.014639,1.000000,0.123709,0.013922,0.019019,0.025259
Private,0.123330,-0.092353,0.123709,1.000000,-0.173587,-0.048557,-0.169160
Public,0.172740,-0.142811,0.013922,-0.173587,1.000000,-0.061295,-0.049959
Research,0.030260,-0.026772,0.019019,-0.048557,-0.061295,1.000000,-0.053085
University,0.078119,-0.072414,0.025259,-0.169160,-0.049959,-0.053085,1.000000


In [8]:
from pgmpy.independencies import Independencies
# build skeleton from list of independencies:
year_pairs = []
for year1 in years:
    for year2 in years:
        if year1 != year2:
            year_pairs.append([year1, year2])
#country_pairs = []
#for country1 in countries:
#    for country2 in countries:
#        if country1 != country2:
#            country_pairs.append([country1, country2])
            

#pairs = country_pairs + year_pairs
pairs = year_pairs
ind = Independencies()
for pair in pairs:
    ind.add_assertions(pair)
ind.closure()
# we need to compute closure, otherwise this set of independencies doesn't
# admit a faithful represen

(1997 ⟂ 1999)
(2004 ⟂ 2010)
(2008 ⟂ 2014)
(1991 ⟂ 2008)
(1993 ⟂ 2009)
(2000 ⟂ 2015)
(2006 ⟂ 2008)
(1991 ⟂ 1994)
(2001 ⟂ 2006)
(1997 ⟂ 2011)
(1987 ⟂ 2004)
(1999 ⟂ 2004)
(1987 ⟂ 2020)
(2005 ⟂ 2008)
(2003 ⟂ 2005)
(1987 ⟂ 2006)
(2012 ⟂ 2017)
(2001 ⟂ 2011)
(1989 ⟂ 2002)
(1987 ⟂ 1993)
(1999 ⟂ 2000)
(1994 ⟂ 2019)
(2012 ⟂ 2021)
(1993 ⟂ 1999)
(2009 ⟂ 2020)
(1993 ⟂ 2012)
(1987 ⟂ 2003)
(1993 ⟂ 1995)
(1999 ⟂ 2005)
(1991 ⟂ 2020)
(2000 ⟂ 2019)
(2004 ⟂ 2016)
(2013 ⟂ 2017)
(2010 ⟂ 2019)
(2008 ⟂ 2021)
(2001 ⟂ 2021)
(1990 ⟂ 2012)
(2003 ⟂ 2004)
(2014 ⟂ 2020)
(1995 ⟂ 2010)
(1994 ⟂ 2003)
(2003 ⟂ 2012)
(2000 ⟂ 2011)
(1989 ⟂ 2011)
(2005 ⟂ 2009)
(1995 ⟂ 2016)
(2011 ⟂ 2013)
(2004 ⟂ 2008)
(1994 ⟂ 2016)
(2005 ⟂ 2010)
(1991 ⟂ 1993)
(1995 ⟂ 2011)
(2002 ⟂ 2021)
(1994 ⟂ 2009)
(2008 ⟂ 2011)
(2002 ⟂ 2007)
(2002 ⟂ 2004)
(2004 ⟂ 2011)
(2002 ⟂ 2020)
(1994 ⟂ 2006)
(2004 ⟂ 2007)
(2004 ⟂ 2012)
(2006 ⟂ 2014)
(1991 ⟂ 2014)
(1995 ⟂ 2006)
(1989 ⟂ 2001)
(2002 ⟂ 2018)
(1991 ⟂ 2019)
(2011 ⟂ 2017)
(2005 ⟂ 2019)
(1987 ⟂ 1995)
(2003 

In [9]:
from pgmpy.estimators import PC
c = PC(data[dag_keys].fillna(0))

model = c.estimate(return_type = "dag", significance_level = 0.01, 
                   max_cond_vars = max_cond_vars, ci_test = "chi_square")
model.__dict__

Working for n conditional variables: 3:  50%|██████████████████████                      | 3/6 [00:00<00:00,  4.80it/s]


{'graph_attr_dict_factory': dict,
 'node_dict_factory': dict,
 'node_attr_dict_factory': dict,
 'adjlist_outer_dict_factory': dict,
 'adjlist_inner_dict_factory': dict,
 'edge_attr_dict_factory': dict,
 'graph': {},
 '_node': {'Private': {'weight': None},
  'Num_Citations': {'weight': None},
  'Research': {'weight': None}},
 '_adj': {'Private': {'Num_Citations': {'weight': None}},
  'Num_Citations': {},
  'Research': {'Num_Citations': {'weight': None}}},
 '_pred': {'Private': {},
  'Num_Citations': {'Private': {'weight': None}, 'Research': {'weight': None}},
  'Research': {}},
 '_succ': {'Private': {'Num_Citations': {'weight': None}},
  'Num_Citations': {},
  'Research': {'Num_Citations': {'weight': None}}},
 'latents': set()}

In [10]:
sig_corr_dct

{'Current Impact_Factor': [('Num_Citations', 0.216), ('Public', 0.194)],
 'Num_Citations': [('Current Impact_Factor', 0.216), ('Private', 0.126)],
 'Public': [('Current Impact_Factor', 0.194),
  ('Private', -0.166),
  ('International', -0.149)],
 'Private': [('Num_Citations', 0.126),
  ('Public', -0.166),
  ('University', -0.169)],
 'University': [('Private', -0.169)],
 'International': [('Public', -0.149)],
 'Research': []}

In [11]:
pcs_dct

{'Current Impact_Factor': {'Current Impact_Factor': 1,
  'Num_Citations':            n      r        CI95%  p-val
  pearson  503  0.216  [0.13, 0.3]    0.0,
  'Public':            n      r         CI95%  p-val
  pearson  503  0.194  [0.11, 0.28]    0.0,
  'Private':            n      r         CI95%  p-val
  pearson  503  0.108  [0.02, 0.19]  0.015,
  'University':            n      r          CI95%  p-val
  pearson  503  0.082  [-0.01, 0.17]  0.066,
  'International':            n      r        CI95%  p-val
  pearson  503  0.112  [0.02, 0.2]  0.013,
  'Research':            n      r          CI95%  p-val
  pearson  503  0.033  [-0.06, 0.12]  0.469},
 'Num_Citations': {'Current Impact_Factor':            n      r        CI95%  p-val
  pearson  503  0.216  [0.13, 0.3]    0.0,
  'Num_Citations': 1,
  'Public':            n      r         CI95%  p-val
  pearson  503  0.012  [-0.08, 0.1]  0.791,
  'Private':            n      r         CI95%  p-val
  pearson  503  0.126  [0.04, 0.21]  0.00